In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing as pre
from sklearn.metrics import classification_report,confusion_matrix


In [2]:
#load the data
train_pd = pd.read_csv("train.csv")
test_pd = pd.read_csv("test.csv")


In [3]:
#Id Column is not required 
X_before_train = train_pd.drop(['id','time'],axis=1)
y_before_train = train_pd['time']

x_to_be_preicted = test_pd.drop('id',axis=1)

In [4]:
#Hot encoding for  penalty column
train_pd_encoded = pd.get_dummies(X_before_train, columns=['penalty'])

#testing data
test_pd_encoded = pd.get_dummies(x_to_be_preicted, columns=['penalty'])

In [5]:
train_pd_encoded.head(5)

,l1_ratio,alpha,max_iter,random_state,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,n_informative,flip_y,scale,penalty_elasticnet,penalty_l1,penalty_l2,penalty_none
0,0.304083,0.0001,417,475,-1,1089,327,4,3,7,0.074798,24.242009,0,0,0,1
1,0.727744,0.0010,578,569,1,790,373,4,5,7,0.077781,54.626302,0,1,0,0
2,0.745885,0.0100,588,529,2,428,1198,2,5,6,0.030196,17.999964,0,0,0,1
3,0.474605,0.0010,829,103,4,877,313,6,5,7,0.057261,82.257222,0,0,0,1
4,0.395049,0.0010,167,418,2,216,644,8,5,11,0.073728,95.515601,1,0,0,0


In [6]:
train_pd_encoded['feature_sample'] = train_pd_encoded['n_features'] * train_pd_encoded['n_samples']
train_pd_encoded.drop(['n_features','n_samples'],inplace=True,axis=1)

In [7]:
test_pd_encoded['feature_sample'] = test_pd_encoded['n_features'] * test_pd_encoded['n_samples']
test_pd_encoded.drop(['n_features','n_samples'],inplace=True,axis=1)

In [8]:
#n_jobs -1 is the max value so we need to make it max of the list of values.

test_pd_encoded.loc[test_pd_encoded['n_jobs']==-1,'n_jobs'] = test_pd_encoded.n_jobs.max()+1
train_pd_encoded.loc[train_pd_encoded['n_jobs']==-1,'n_jobs'] = train_pd_encoded.n_jobs.max()+1

In [9]:
train_pd_encoded['n_classes'] = train_pd_encoded.n_classes.astype('category')
train_pd_encoded['n_clusters_per_class'] = train_pd_encoded.n_clusters_per_class.astype('category')
train_pd_encoded['n_informative'] = train_pd_encoded.n_informative.astype('category')
#make the data type as float
train_pd_encoded['max_iter'] = train_pd_encoded.max_iter.astype(float)
train_pd_encoded['random_state'] = train_pd_encoded.random_state.astype(float)

#testing data
test_pd_encoded['n_classes'] = test_pd_encoded.n_classes.astype('category')
test_pd_encoded['n_clusters_per_class'] = test_pd_encoded.n_clusters_per_class.astype('category')
test_pd_encoded['n_informative'] = test_pd_encoded.n_informative.astype('category')
#make the data type as float
test_pd_encoded['max_iter'] = test_pd_encoded.max_iter.astype(float)
test_pd_encoded['random_state'] = test_pd_encoded.random_state.astype(float)


In [10]:
scaler = pre.StandardScaler()
#scaler = pre.RobustScaler()

In [11]:
test_pd_encoded['alpha'] = test_pd_encoded.alpha.astype('category')
train_pd_encoded['alpha'] = train_pd_encoded.alpha.astype('category')

train_pd_encoded['n_jobs'] = train_pd_encoded.n_jobs.astype('category')
test_pd_encoded['n_jobs'] = test_pd_encoded.n_jobs.astype('category')

In [12]:
train_pd_encoded.head(2)

,l1_ratio,alpha,max_iter,random_state,n_jobs,n_classes,n_clusters_per_class,n_informative,flip_y,scale,penalty_elasticnet,penalty_l1,penalty_l2,penalty_none,feature_sample
0,0.304083,0.0001,417.0,475.0,9,4,3,7,0.074798,24.242009,0,0,0,1,356103
1,0.727744,0.0010,578.0,569.0,1,4,5,7,0.077781,54.626302,0,1,0,0,294670


In [13]:
#model train test 
X_train, X_test, y_train, y_test = train_test_split(train_pd_encoded, y_before_train,test_size=0.25, random_state=0)

In [14]:
mlp = MLPRegressor(solver='lbfgs', 
                   activation='relu',
                   hidden_layer_sizes=(8,8), 
                   max_iter=1000,
                   learning_rate='constant', 
                   learning_rate_init=1,
                   alpha=10,
                   random_state=58)

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaled_to_be_predicted =  scaler.transform(test_pd_encoded)

mlp.fit(X_train_scaled,y_train)

print("Testing set score: %f" % mlp.score(X_test_scaled, y_test))
print("Training set score: %f" % mlp.score(X_train_scaled, y_train))

final_predicted = mlp.predict(scaled_to_be_predicted)
final_predicted

Testing set score: 0.875956
Training set score: 0.989281


array([ 1.57159638, 15.14941498,  0.30022846,  2.41776974,  8.02467734,
        7.72007247,  3.59200967,  0.51478193, 14.16357516,  0.33292911,
        8.03547922,  6.02758302,  0.3992456 , 31.25027415,  0.35392951,
        0.88051138,  0.9968097 , 15.09506171,  3.76776247,  1.1425106 ,
        0.43603279,  0.46218871,  0.28226729,  0.83067999,  1.23898779,
        3.99595781,  1.54233715,  2.54973215,  3.38455506,  3.15432402,
       19.79814546,  1.87970757,  0.28371907, 10.9452285 ,  5.73496831,
        0.51630897,  2.89618191,  2.36496819,  0.81783566,  1.11612249,
        5.28401503, 25.5111477 ,  1.1234557 ,  5.16514122,  0.97917709,
        3.13360659,  2.57877136,  1.52119165,  9.37555289,  0.60341301,
        0.35772003, 24.39264101,  0.3200499 ,  5.9624891 , 21.7367936 ,
        0.39269361,  1.18314843,  0.63795452,  0.50110599,  2.52616819,
        5.96553827,  6.62322341,  0.3408883 ,  6.78551905,  3.94312044,
        8.23181394,  2.19140362,  0.2724529 ,  7.21946621,  0.70

In [15]:
##predicted has some -ve values, so use the mean values
final_predict_df = pd.DataFrame({'Time' : (final_predicted)})
final_predict_df.index.name = 'Id'
predicted_mean = final_predicted.mean()
final_predict_df[final_predict_df<0] = 0
final_predict_df.to_csv('submit_final.csv')
